# Generation of files for media preparation using biomek

This notebook generates several files used in an automated media optimization project. Given a file with stock concentrations and a file with target concentrations (recommended for example by ART), this notebooks generates transfer volumes needed to achieve those target concentrations. Transfer volumes are provided as a table for all components and destination wells, but also as files required for a Biomek run. The notebook also generates a file with instructions on how to prepare a plate with stock solutions.

## Inputs and outputs

#### Required files to run this notebook:
   - `../data/flaviolin/standard_recipe_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/flaviolin/stock_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/DBTLX/target_concentrations.csv` - this is an output from an ART run and it will change at every DBTL cycle

   

#### Files generated by running this notebook:


   - `stock_plate.csv` - instructions on how to prepare the source plate
   
   - `dest_volumes.csv` - volumes of all components in each of the destination wells
   
   
   
The files that need to be uploaded to the Biomek, following this particular order:
   
   - `P200_water.csv`
   
   - `P20_water.csv`
         
   - `P20_kan.csv` 
   
   - `P200_components.csv` 
   
   - `P20_components.csv` 
  
   - `P20_culture.csv` 
   
   
Note that all these different files are needed as biomek needs a separate one for different types of source/destination plate and tips. Also, it cannot follow the order of operations within a file so we have to enforce an order using different files (e.g. first a file with kan transfers, then a file with components transfers).
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [38]:
import os
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from core import find_volumes, test_volumes


## User parameters

In [39]:
CYCLE = 2

user_params = {
    'stock_conc_file': '../data/flaviolin/stock_concentrations.csv',
    'standard_media_file': '../data/flaviolin/standard_recipe_concentrations.csv',
    'target_conc_file': f'../data/flaviolin/DBTL{CYCLE}/target_concentrations.csv',
    'output_path': f'../data/flaviolin/DBTL{CYCLE}',  # Path for output files
    'well_volume': 1500,            # Total volume of the media content in the destination well
    'tips': ['f20', 'f200'],        # Choose available tips from f20, s20, f50, s50, f200, s200
    'min_transfer_volume': 5.,      # Minimal transfer volume of the liquid handler
    'culture_factor': 100           # Dilution factor for culture, e.g. 100x, 1000x

}

Setup tips

In [40]:
df_tips = pd.DataFrame([['f20', 36],
                        ['s20', 72],
                        ['f50', 45],
                        ['s50', 81],
                        ['f200', 135],
                        ['s200', 171]],
                       columns=['Tips', 'Max Volume'],
                       ).set_index('Tips')


In [41]:
max_tip_volume = max([df_tips.loc[tip]['Max Volume'] for tip in user_params['tips']])
max_tip_volume

135

Load the standard media recipe

In [42]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

In [43]:
df_stand

,Concentration[mM],Solubility[mM]
Component,,
MOPS,40.00000,2389.37
Tricine,4.00000,500.08
H3BO3,0.00400,700.00
Glucose,20.00000,5045.63
K2SO4,0.29000,636.98
K2HPO4,1.32000,8564.84
FeSO4,0.01000,1645.73
NH4Cl,9.52000,6543.28
MgCl2,0.52000,569.27


Load the stock concentrations

In [44]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")

In [45]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0,2000.0,1.0
Tricine,400.0,400.0,1.0
H3BO3,0.12,2.4,20.0
Glucose,3000.0,3000.0,1.0
K2SO4,8.7,43.5,5.0
K2HPO4,79.2,396.0,5.0
FeSO4,0.3,6.0,20.0
NH4Cl,1904.0001,1904.0001,1.0
MgCl2,15.6,15.6,1.0


## Create stock solutions plate dataframe


In [46]:
df_stock_plate = pd.DataFrame(
    columns=["Component", "Stock", "Concentration[mM]"])
    
i = 0
num_comp = len(df_stock)
for i in range(2*num_comp):

    if i < num_comp:
        component = df_stock.index[i]
        df_stock_plate.loc[i] = [
            component, 
            "high",
            df_stock.loc[component]["High Concentration[mM]"]
        ]
    elif i < 2*num_comp:
        component = df_stock.index[(i-num_comp)]
        if df_stock.loc[component]["Dilution Factor"] > 1.0:
            df_stock_plate.loc[i] = [
                component, 
                "low",
                df_stock.loc[component]["Low Concentration[mM]"]
            ]
            
df_stock_plate.reset_index(drop=True, inplace=True)

In [47]:
df_stock_plate

,Component,Stock,Concentration[mM]
0,MOPS,high,2000.0
1,Tricine,high,400.0
2,H3BO3,high,2.4
3,Glucose,high,3000.0
4,K2SO4,high,43.5
5,K2HPO4,high,396.0
6,FeSO4,high,6.0
7,NH4Cl,high,1904.0001
8,MgCl2,high,15.6
9,NaCl,high,1500.0


We will not save this stock plate for now, as likely not all wells/concentration levels will be needed once we load the target concentrations.

Reformat values from string to float:

In [48]:
df_stock.loc['Kan'] = [300., 300., 1.]
df_stock["High Concentration[mM]"] = df_stock["High Concentration[mM]"].astype(float)
df_stock["Low Concentration[mM]"] = df_stock["Low Concentration[mM]"].astype(float)

In [49]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0000,2000.0000,1.0
Tricine,400.0000,400.0000,1.0
H3BO3,0.1200,2.4000,20.0
Glucose,3000.0000,3000.0000,1.0
K2SO4,8.7000,43.5000,5.0
K2HPO4,79.2000,396.0000,5.0
FeSO4,0.3000,6.0000,20.0
NH4Cl,1904.0001,1904.0001,1.0
MgCl2,15.6000,15.6000,1.0


## Read target concentrations

Read ART suggested target concentrations. Note that those are only for components which are being explored (not those that are kept constant). 

In [50]:
target_conc_file = user_params['target_conc_file']
df_target_conc = pd.read_csv(target_conc_file, index_col=0)
df_target_conc.head()

,H3BO3,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,
A1,0.026992,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A2,0.026992,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A3,0.026992,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A4,0.026992,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172
A5,0.001199,2.356433,0.965094,0.005574,10.252992,0.428695,133.073471,0.000090,0.000246,0.000706,0.002361,0.000076


### Add fixed components and antibiotic concentrations to the target concentrations dataframe

Find fixed components as those from the standard recipe that are not listed in target concentration file, which is an output from ART

In [51]:
comp_fixed = list(df_stand.drop(df_target_conc.columns).index)
print('Fixed components: ')
for comp in comp_fixed:
    df_target_conc[comp] = df_stand.at[comp, 'Concentration[mM]']
    print(f'{comp}')

Fixed components: 
MOPS
Tricine
Glucose
Kan


Make sure the order of columns is the same as in the stock dataframe:

In [52]:
columns = df_stock.index
df_target_conc = df_target_conc[columns]

In [53]:
df_target_conc.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172,1.0
A2,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172,1.0
A3,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172,1.0
A4,40.0,4.0,0.026992,20.0,0.229277,0.556502,0.002319,7.370092,1.386888,355.093844,0.000028,0.000101,0.000790,0.000560,0.000172,1.0
A5,40.0,4.0,0.001199,20.0,2.356433,0.965094,0.005574,10.252992,0.428695,133.073471,0.000090,0.000246,0.000706,0.002361,0.000076,1.0


Save a file with a complete list of components concentrations, describing the media (this file will lated be used to create experiment description file for EDD import).

In [54]:
final_conc_file = f"{user_params['output_path']}/media_descriptions.csv"
df_target_conc.drop(columns='Kan').to_csv(final_conc_file)

## Calculate all transfer volumes

Also, create a dataframe with levels of stock concentrations needed to achieve those volumes, which will indicate from which source well the transfer should be made.

In [55]:
verbose = 0
EPS = 0.000001
min_tip_volume = user_params['min_transfer_volume']

df_conc_level = pd.DataFrame(data='high',
    index=df_target_conc.index,
    columns=df_target_conc.columns)

df_volumes = df_target_conc.copy()

# Add column for water
df_volumes['Water'] = None

for i in range(len(df_volumes)):
    volumes, df = find_volumes(
        user_params['well_volume'], 
        components=df_stock.index,
        stock_conc_val=df_stock["High Concentration[mM]"].values, 
        target_conc_val=df_target_conc.iloc[i].values,
        culture_ratio=user_params['culture_factor']
    )
    df_volumes.iloc[i] = volumes
    
    # Find volumes smaller than min transfer volume
    comp_small_vol = df_volumes.iloc[i][
        df_volumes.iloc[i] < min_tip_volume - EPS
    ].index
    if verbose:
        print(f'Compoments small: {comp_small_vol}')
    
    # Assign low concentrations for those components
    stock_new = df_stock["High Concentration[mM]"].copy()
    for comp in comp_small_vol:
        stock_new[comp] = df_stock.loc[comp]["Low Concentration[mM]"]
        df_conc_level.iloc[i][comp] = 'low'

    # Recalculate  volumes
    try:
        volumes, df = find_volumes(
            user_params['well_volume'], 
            components=df_stock.index,
            stock_conc_val=stock_new.values, 
            target_conc_val=df_target_conc.iloc[i].values,
            culture_ratio=user_params['culture_factor']
        )
        df_volumes.iloc[i] = volumes
    except:
        print('High + Low failed')
    
display(df_volumes.head(5))


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water
Well,,,,,,,,,,,,,,,,,
A1,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137
A2,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137
A3,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137
A4,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137
A5,30.0,15.0,14.989318,10.0,81.256322,18.278296,27.869006,8.077462,41.220638,133.073471,7.517715,40.926155,17.643461,7.378641,37.879023,5.0,988.890494


In [56]:
df_conc_level.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
Well,,,,,,,,,,,,,,,,
A1,high,high,high,high,high,low,low,high,high,high,low,low,high,low,low,high
A2,high,high,high,high,high,low,low,high,high,high,low,low,high,low,low,high
A3,high,high,high,high,high,low,low,high,high,high,low,low,high,low,low,high
A4,high,high,high,high,high,low,low,high,high,high,low,low,high,low,low,high
A5,high,high,low,high,high,low,low,high,high,high,high,low,high,high,low,high


Add volumes for culture

In [57]:
df_volumes['Culture'] = user_params['well_volume'] / user_params['culture_factor']
df_volumes.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water,Culture
Well,,,,,,,,,,,,,,,,,,
A1,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137,15.0
A2,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137,15.0
A3,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137,15.0
A4,30.0,15.0,16.869858,10.0,7.906088,10.539811,11.596698,5.806269,133.354632,355.093844,46.661961,16.899018,19.740620,35.026078,85.890987,5.0,679.614137,15.0
A5,30.0,15.0,14.989318,10.0,81.256322,18.278296,27.869006,8.077462,41.220638,133.073471,7.517715,40.926155,17.643461,7.378641,37.879023,5.0,988.890494,15.0


Check if sum of all volumes is equal to total well volume

In [58]:
assert (np.sum(df_volumes.values, axis=1) == user_params['well_volume']).all(), 'Sum of all volumes is not equal to total well volume!' 

### Save volumes in the destination wells to a file

In [59]:
volumes_file = f"{user_params['output_path']}/dest_volumes.csv"
df_volumes.to_csv(volumes_file)

## Create source plate

### Calculate minimal volumes needed in the source wells

In [60]:
tot_vol_water = np.sum(df_volumes['Water'].values)
print(f'Total volume of water needed: {tot_vol_water:.0f} uL + dead volume')

Total volume of water needed: 46759 uL + dead volume


In [61]:
for i in range(len(df_stock_plate)):
    comp = df_stock_plate.iloc[i]['Component']
    stock_level = 'high'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate.loc[i, 'Volume [uL]'] = np.round(tot_vol_comp)
    
# Add culture to the stock plate
tot_vol_culture = np.sum(df_volumes['Culture'].values) 
culture = {'Component': 'Culture', 
           'Stock': None,
           'Concentration[mM]': None,
           'Volume [uL]': np.round(tot_vol_culture)
          }
df_stock_plate = df_stock_plate.append(culture, ignore_index = True)
df_stock_plate


,Component,Stock,Concentration[mM],Volume [uL]
0,MOPS,high,2000.0,1440.0
1,Tricine,high,400.0,720.0
2,H3BO3,high,2.4,264.0
3,Glucose,high,3000.0,480.0
4,K2SO4,high,43.5,1889.0
5,K2HPO4,high,396.0,245.0
6,FeSO4,high,6.0,222.0
7,NH4Cl,high,1904.0001,394.0
8,MgCl2,high,15.6,4097.0
9,NaCl,high,1500.0,5449.0


Define the source plate:

In [62]:
num_source_wells = len(df_stock_plate)

if num_source_wells <= 24:
    well_volume = 9000  # including dead volume
    dead_volume = 50
else:
    well_volume = 4000  # including dead volume
    dead_volume = 30


Are the total volumes smaller than well volume of the plate?

In [64]:
df_new_wells = pd.DataFrame(columns=df_stock_plate.columns)
ind_drop = []
num_all_wells = len(df_stock_plate)
for i in range(len(df_stock_plate)):
    tot_volume = df_stock_plate.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate.iloc[i]['Component']
        conc_level = df_stock_plate.iloc[i]['Stock']
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {
             'Component' : pd.Series(comp, index=indices),
             'Stock': conc_level,
             'Concentration[mM]': pd.Series(df_stock_plate.iloc[0]['Concentration[mM]'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate.at[i,'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate.drop(ind_drop, inplace=True)
df_stock_plate = df_stock_plate.append(df_new_wells, ignore_index=False)
df_stock_plate = df_stock_plate.sort_index().reset_index(drop=True) 


........We need 2 well(s) for MgCl2 with high concentration
We need 2 well(s) for NaCl with high concentration
................Finished.


In [65]:
df_stock_plate

,Component,Stock,Concentration[mM],Volume [uL]
0,MOPS,high,2000.0,1500.0
1,Tricine,high,400.0,780.0
2,H3BO3,high,2.4,324.0
3,Glucose,high,3000.0,540.0
4,K2SO4,high,43.5,1949.0
5,K2HPO4,high,396.0,305.0
6,FeSO4,high,6.0,282.0
7,NH4Cl,high,1904.0001,454.0
8,MgCl2,high,2000.0,4000.0
9,NaCl,high,2000.0,4000.0


Assign well names:

In [67]:
num_source_wells = len(df_stock_plate) 

if num_source_wells <= 24:
    source_well_type = '24-well'
    print(f'Use {source_well_type} source plate')
    well_rows = 'ABCD'
    well_columns = '123456'
else:
    source_well_type = '48-well'
    print(f'Use {source_well_type} source plate')
    well_rows = 'ABCDEFGH'
    well_columns = '123456'
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate.reset_index(drop=True, inplace=True)
df_stock_plate['Well'] = well_names
df_stock_plate = df_stock_plate.set_index(['Well'])
df_stock_plate

Use 48-well source plate


,Component,Stock,Concentration[mM],Volume [uL]
Well,,,,
A1,MOPS,high,2000.0,1500.0
B1,Tricine,high,400.0,780.0
C1,H3BO3,high,2.4,324.0
D1,Glucose,high,3000.0,540.0
E1,K2SO4,high,43.5,1949.0
F1,K2HPO4,high,396.0,305.0
G1,FeSO4,high,6.0,282.0
H1,NH4Cl,high,1904.0001,454.0
A2,MgCl2,high,2000.0,4000.0


In [38]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate.csv"
df_stock_plate.to_csv(stock_plate_file)

## Define transfers for biomek

In [39]:
column_names = [
    "Source Position",
    "Source Well",
    "Destination Position",
    "Destination Well",
    "Transfer Volume [uL]"
]

### Create water transfers

In [40]:
P20_water = pd.DataFrame(columns=column_names)
P200_water = P20_water.copy()

i = 0  # counter for index

comp = 'Water'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_water.at[i, "Destination Well"] = dest_well
        P20_water.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_water.at[i, "Destination Well"] = dest_well
            P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_water["Source Position"] = "P3" 
P20_water["Source Well"] = "A1"  # reservoir plate
P20_water["Destination Position"] = "P2" 
P200_water["Source Position"] = "P3" 
P200_water["Source Well"] = "A1"
P200_water["Destination Position"] = "P2" 

### Create kan transfers

In [41]:
P20_kan = pd.DataFrame(columns=column_names)
P200_kan = P20_kan.copy()

i = 0  # counter for index

comp = 'Kan'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_kan.at[i, "Destination Well"] = dest_well
        P20_kan.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_kan.at[i, "Destination Well"] = dest_well
            P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_kan["Source Position"] = "P1" 
P20_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P20_kan["Destination Position"] = "P2"
P200_kan["Source Position"] = "P1" 
P200_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P200_kan["Destination Position"] = "P2"


### Create culture transfers

Note that we assume here the volumes for culture are small enough for p20 pipette, if that is not the case you need to adjust.

In [42]:
P20_culture = pd.DataFrame(columns=column_names)

comp = 'Culture'
vol = df_volumes[comp][0]
    
for i, dest_well in enumerate(df_volumes.index):
    
    P20_culture.at[i, "Destination Well"] = dest_well
    P20_culture.at[i, "Transfer Volume [uL]"] = vol

P20_culture["Source Position"] = "P4" 
P20_culture["Source Well"] = df_stock_plate_low[
    df_stock_plate_low["Component"]==comp
].index[0]  
P20_culture["Destination Position"] = "P2" 


### Create component transfers

Create a column to track the current volume in the wells of the source plate:

In [43]:
df_stock_plate['Current Volume'] = well_volume


In [44]:
P20_components = pd.DataFrame(columns=column_names) 
P200_components = P20_components.copy()

In [45]:
i = 0  # counter for index

components = list(df_volumes.columns.drop(['Kan', 'Water', 'Culture']))

for comp in components:
    
    for dest_well in df_volumes.index:
        vol = df_volumes.at[dest_well, comp]
        conc_level = df_conc_level.at[dest_well, comp]
        
        # Find the source well in the stock plate with enough volume 
        # Find all wells with this component
        source_wells = df_stock_plate[
            (df_stock_plate["Component"]==comp)
        ].index
        
        # Find indices where these wells have volume larger than transfer volume
        ind_vol = np.where(df_stock_plate.loc[source_wells, 'Current Volume']  > vol + dead_volume)[0]
        # Assign as the source well the first one with enough volume
        source_well = source_wells[ind_vol[0]]      
        
        # P20 transfer
        if vol < 30:
            P20_components.at[i, "Source Well"] = source_well
            P20_components.at[i, "Destination Well"] = dest_well
            P20_components.at[i, "Transfer Volume [uL]"] = vol
            i += 1

        # P200 transfer
        else:
            # Divide the transfer in parts until whole volume is transfered
            # How many transfers
            num_transfers = int(np.ceil(vol / max_tip_volume))
            transf_vol = vol / num_transfers
            for trans_num in range(num_transfers):
                P200_components.at[i, "Source Well"] = source_well
                P200_components.at[i, "Destination Well"] = dest_well
                P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                i += 1
                    
        # Update the current volume in the source well
        df_stock_plate.loc[source_well, 'Current Volume'] -= vol

P20_components["Source Position"] = "P1" 
P200_components["Source Position"] = "P1"
P20_components["Destination Position"] = "P2" 
P200_components["Destination Position"] = "P2" 

In [46]:
P20_components.head(5)

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
12,P1,A1,P2,B5,30.0
13,P1,A1,P2,B6,30.0
14,P1,A1,P2,B7,30.0
15,P1,A1,P2,B8,30.0
16,P1,A1,P2,C1,30.0


## Calculate number of transfers and tips needed

In [47]:
num_transf_water_p200 = len(P200_water)
num_transf_water_p20 = len(P20_water)
num_transf_comp_p20 = len(P20_components)
num_transf_comp_p200 = len(P200_components)
num_transf_culture = len(P20_culture)
num_transf_kan_p20 = len(P20_kan)
num_transf_kan_p200 = len(P200_kan)
num_mixing = 1  # mixing the culture well 
num_total_transf = num_transf_water_p200 + num_transf_water_p20 + num_transf_comp_p20 + num_transf_comp_p200+num_transf_kan_p20+num_transf_kan_p200+num_transf_culture+num_mixing
print(f'Number of transfers:')
print(f'\t Water (p200): {num_transf_water_p200}')
print(f'\t Water (p20): {num_transf_water_p20}')
print(f'\t Kan (p20): {num_transf_kan_p20}')
print(f'\t Kan (p200): {num_transf_kan_p200}')
print(f'\t Components (p200): {num_transf_comp_p200}')
print(f'\t Components (p20): {num_transf_comp_p20}')
print(f'\t Culture (p20): {num_transf_culture}')
print(f'\t Mixing (p20): {num_mixing}')
print(f'Total number of transfers: {num_total_transf}')

Number of transfers:
	 Water (p200): 376
	 Water (p20): 0
	 Kan (p20): 48
	 Kan (p200): 0
	 Components (p200): 264
	 Components (p20): 492
	 Culture (p20): 48
	 Mixing (p20): 1
Total number of transfers: 1229


Calculate number of tip boxes needed:

In [48]:
print(f'This protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_comp_p20 + num_transf_kan_p20 + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')


This protocol requires:
	3 box(es) of p200 tips
		2 box(es) + 79 tips
	7 box(es) of p20 tips
		6 box(es) + 13 tips


Introduce a pause in the biomek method after p200 components transfers.

In [49]:
print(f'The first part of the protocol requires:')
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_kan_p20 

full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The first part of the protocol requires:
	3 box(es) of p200 tips
		2 box(es) + 79 tips
	1 box(es) of p20 tips
		0 box(es) + 48 tips


For the next step we need:

In [50]:
print(f'The second part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_20 = num_transf_comp_p20  + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

This protocol requires:
	6 box(es) of p20 tips
		5 box(es) + 61 tips


### Save biomek files

In [51]:
biomek_files_dir = f"{user_params['output_path']}/biomek_files/"
os.makedirs(biomek_files_dir, exist_ok=True)

P200_water_file = f"{biomek_files_dir}/P200_water.csv"
P20_water_file = f"{biomek_files_dir}/P20_water.csv"
P20_kan_file = f"{biomek_files_dir}/P20_kan.csv"
P200_components_file = f"{biomek_files_dir}/P200_components.csv"
P20_components_file = f"{biomek_files_dir}/P20_components.csv"
P20_culture_file = f"{biomek_files_dir}/P20_culture.csv"

P200_water.to_csv(P200_water_file, index=False)
P20_water.to_csv(P20_water_file, index=False)
P20_kan.to_csv(P20_kan_file, index=False)
P200_components.to_csv(P200_components_file, index=False)
P20_components.to_csv(P20_components_file, index=False)
P20_culture.to_csv(P20_culture_file, index=False)